<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Tic-tac-toe-with-Q-learning" data-toc-modified-id="Tic-tac-toe-with-Q-learning-1">Tic-tac-toe with Q-learning</a></span></li><li><span><a href="#Learning-Outcomes" data-toc-modified-id="Learning-Outcomes-2">Learning Outcomes</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#By-the-end-of-this-session,-you-should-be-able-to:" data-toc-modified-id="By-the-end-of-this-session,-you-should-be-able-to:-2.0.1">By the end of this session, you should be able to:</a></span></li></ul></li></ul></li><li><span><a href="#Tic-tac-toe-For-The-Win-" data-toc-modified-id="Tic-tac-toe-For-The-Win--3">Tic-tac-toe For The Win </a></span></li><li><span><a href="#-The-Code" data-toc-modified-id="-The-Code-4"> The Code</a></span></li><li><span><a href="#Summary" data-toc-modified-id="Summary-5">Summary</a></span></li></ul></div>

<center><h2>Tic-tac-toe with Q-learning</h2></center>

<center><h2>Learning Outcomes</h2></center>

#### By the end of this session, you should be able to:

- Frame tic-tac-toe as a Reinforcement Learning problem.
- Create an agent that learns an optimal tic-tac-toe policy through Q-learning.
- Write Q-learning from scratch.

Tic-tac-toe For The Win 
-----
 
1. Please start by reading the [Wikipedia article on tic-tac-toe](https://en.wikipedia.org/wiki/Tic-tac-toe) to make sure you understand the game.

1. Play a couple of games against a friend to understand the game play.

1. Re-read Sutton and Barto's 1.5 "An Extended Example: Tic-Tac-Toe"

Sample state tree:

<center><img src="images/tic-tac-toe-tree.png" width="75%"/></center>


For tic-tac-toe, answer the following questions in the following cell:

1. How is it sequential?
1. What is the environment? 
1. Who is the agent?
1. What are the states?
1. What are the rewards?

1 point

YOUR ANSWER HERE

 The Code
 ------

In [ ]:
reset -fs

In [ ]:
from collections import Counter
import random

In [ ]:
class TicTacToe():
    def __init__(self):
        self.state = '         ' # Represent state as a vector of strings (not a matrix)
        self.player = 'X'
        self.winner = None

    def allowed_moves(self):
        states = []
        if self.winner:
            return states
        for i in range(len(self.state)):
            if self.state[i] == ' ':
                states.append(self.state[:i] + self.player + self.state[i+1:])
        return states

    def make_move(self, next_state):
        if self.winner:
            raise(Exception("Game already completed, cannot make another move!"))
        if not self.valid_move(next_state):
            raise(Exception("Cannot make move {} to {} for player {}".format(
                    self.state, next_state, self.player)))

        self.state = next_state
        self.winner = self.predict_winner(self.state)
        if self.winner:
            self.player = None
        elif self.player == 'X':
            self.player = 'O'
        else:
            self.player = 'X'

    def playable(self):
        return ( (not self.winner) and any(self.allowed_moves()) )

    def predict_winner(self, state):
        lines = [(0,1,2), (3,4,5), (6,7,8), (0,3,6), (1,4,7), (2,5,8), (0,4,8), (2,4,6)]
        winner = None
        for line in lines:
            line_state = state[line[0]] + state[line[1]] + state[line[2]]
            if line_state == 'XXX':
                winner = 'X'
            elif line_state == 'OOO':
                winner = 'O'
        return winner

    def valid_move(self, next_state):
        allowed_moves = self.allowed_moves()
        if any(state == next_state for state in allowed_moves):
            return True
        return False

    def print_board(self):
        s = self.state
        print(f'     {s[0]} | {s[1]} | {s[2]} ')
        print(f'    -----------')
        print(f'     {s[3]} | {s[4]} | {s[5]} ')
        print(f'    -----------')
        print(f'     {s[6]} | {s[7]} | {s[8]} ')

In [ ]:
class Agent():
    "All of the Agent expect Q-learning has been given to you."
    def __init__(self, game_class, epsilon=0.1, alpha=1.0, value_player='X'):
        self.V = dict() # Build up the values of different states as we encounter them; Note the Markov assumption
        self.NewGame = game_class
        self.epsilon = epsilon
        self.alpha = alpha
        self.value_player = value_player

    def state_value(self, game_state):
        "Look up state value. If never seen state, then assume neutral."
        return self.V.get(game_state, 0.0) 

    def learn_game(self, n_episodes=1_000):
        "Let's learn through complete experience to get that reward."
        for episode in range(n_episodes):
            self.learn_from_episode()

    def learn_from_episode(self):
        "Update Values based on reward."
        game = self.NewGame()
        while game.playable():
            self.learn_from_move(game)
        self.V[game.state] = self.reward(game)
        
    def learn_select_move(self, game):
        "Exploration and exploitation"
        allowed_state_values = self.state_values( game.allowed_moves() )
        if game.player == self.value_player:
            best_move = self.argmax_V(allowed_state_values)
        else:
            best_move = self.argmin_V(allowed_state_values)

        selected_move = best_move
        if random.random() < self.epsilon:
            selected_move = self.random_V(allowed_state_values)

        return (best_move, selected_move)

    def play_select_move(self, game):
        "Make the move based on the best option for the player."
        allowed_state_values = self.state_values( game.allowed_moves() )
        if game.player == self.value_player:
            return self.argmax_V(allowed_state_values)
        else:
            return self.argmin_V(allowed_state_values)

    def demo_game(self, verbose=False):
        "Run a game without learning."
        game = self.NewGame()
        t = 0
        while game.playable():
            if verbose:
                print(" \nTurn {}\n".format(t))
                game.print_board()
            move = self.play_select_move(game)
            game.make_move(move)
            t += 1
        if verbose:
            print(" \nTurn {}\n".format(t))
            game.print_board()
        if game.winner:
            if verbose:
                print("\n{} is the winner!".format(game.winner))
            return game.winner
        else:
            if verbose:
                print("\nIt's a draw!")
            return '-'

    def interactive_game(self, agent_player='X'):
        "Play in meatspace."
        game = self.NewGame()
        t = 0
        while game.playable():
            print(" \nTurn {}\n".format(t))
            game.print_board()
            if game.player == agent_player:
                move = self.play_select_move(game)
                game.make_move(move)
            else:
                move = self.request_human_move(game)
                game.make_move(move)
            t += 1

        print(" \nTurn {}\n".format(t))
        game.print_board()

        if game.winner:
            print("\n{} is the winner!".format(game.winner))
            return game.winner
        print("\nIt's a draw!")
        return '-'

    def round_V(self):
        "After training, this makes action selection random from equally-good choices"
        for k in self.V.keys():
            self.V[k] = round(self.V[k],1)

    def state_values(self, game_states):
        return dict((state, self.state_value(state)) for state in game_states)

    def argmax_V(self, state_values):
        "For the best possible states, chose randomly amongst them."
        max_V = max(state_values.values())
        chosen_state = random.choice([state for state, v in state_values.items() if v == max_V])
        return chosen_state

    def argmin_V(self, state_values):
        "For the worst possible states, chose randomly amongst them."
        min_V = min(state_values.values())
        chosen_state = random.choice([state for state, v in state_values.items() if v == min_V])
        return chosen_state

    def random_V(self, state_values):
        "Any state will do."
        return random.choice(list(state_values.keys()))

    def reward(self, game):
        if game.winner == self.value_player:
            return 1.0 # Winning is good
        elif game.winner:
            return -1.0 # Losing is bad
        else:
            return 0.0  # Tying is indifferent

    def request_human_move(self, game):
        "Meatspace"
        allowed_moves = [i+1 for i in range(9) if game.state[i] == ' ']
        human_move = None
        while not human_move:
            idx = int(input('Choose move for {}, from {} : '.format(game.player, allowed_moves)))
            if any([i==idx for i in allowed_moves]):
                human_move = game.state[:idx-1] + game.player + game.state[idx:]
        return human_move


In [ ]:
"""Write the learn_from_move method for Agent class.

Each line has been started for you.
See lecture notes for pseudocode.

No tests (I couldn't figure a fair way of testing).

15 points:
-----

5 points for correctly setting up Q-learning
10 points for Q-learning line
"""

class Agent(Agent): # Note: New class (with same name) inherits from old class (with same name)
    def learn_from_move(self, game):
        "The heart of Q-learning."
        
        current_state = None
        best_next_move, selected_next_move = None, None
        r = None
        
        current_state_value = None
        best_move_value = None
        td_target = None
        self.V[current_state] = None # This is Q-learning. The previous lines setup this line. 
        
        # YOUR CODE HERE
        raise NotImplementedError()
        
        game.make_move(selected_next_move)

In [ ]:
def print_demo_game_stats(agent, n_games=100):
    "Run demo to get sample statistics, no learning"
    long_name = {'X': 'X wins', 
               'O': 'O Wins',
               '-': 'Tie'}
    results = Counter([agent.demo_game() for _ in range(n_games)])
    game_stats = {long_name[k]: v/n_games for k, v in results.items()}
    for k in long_name.values():
        print(f"{k:<7}: {game_stats.get(k, 0):^8.2%}")
    print("")

    
def train(agent, training_block_size = 1_000, n_training_blocks = 10):
    "Given agent, do more training. Return (hopefully) improved agent."
    print("Before learning:")
    print_demo_game_stats(agent, n_games=1_000)
    for n_training_block in range(1, n_training_blocks+1):
        agent.learn_game(training_block_size)
        
        print(f"After {training_block_size*n_training_block:,} learning games:")
        print_demo_game_stats(agent)
    return agent

In [ ]:
# Let's if the agent learns 🤞
agent = Agent(TicTacToe, 
              epsilon = 0.1,
              alpha = 1 
             )
agent = train(agent)

Your training should look something like this:

```
Before learning:
X wins :  56.00% 
O Wins :  31.50% 
Tie    :  12.50% 

After 1,000 learning games:
X wins :  56.00% 
O Wins :  38.00% 
Tie    :  6.00%  

After 2,000 learning games:
X wins :  53.00% 
O Wins :  29.00% 
Tie    :  18.00% 

After 3,000 learning games:
X wins :  46.00% 
O Wins :  34.00% 
Tie    :  20.00% 

After 4,000 learning games:
X wins :  35.00% 
O Wins :  18.00% 
Tie    :  47.00% 

After 5,000 learning games:
X wins :  20.00% 
O Wins :  5.00%  
Tie    :  75.00% 

After 6,000 learning games:
X wins :  16.00% 
O Wins :  3.00%  
Tie    :  81.00% 

After 7,000 learning games:
X wins :  9.00%  
O Wins :  2.00%  
Tie    :  89.00% 

After 8,000 learning games:
X wins :  8.00%  
O Wins :  0.00%  
Tie    :  92.00% 

After 9,000 learning games:
X wins :  2.00%  
O Wins :  1.00%  
Tie    :  97.00% 

After 10,000 learning games:
X wins :  0.00%  
O Wins :  2.00%  
Tie    :  98.00% 
```

In [ ]:
print("Let's peak at first couple of states and their values")
print('State   ', '\t', 'Value')
for k, v in list(agent.V.items())[0:10]:
    print(k, '\t', v),

Your State/Value pairs should look something like this:

```
State    	 Value
None 	 None
          	 0.0
     X    	 0.0
  O  X    	 0.0
  O XX    	 0.0
O O XX    	 1.0
OXO XX    	 1.0
OXOOXX    	 1.0
OXOOXX  X 	 -1.0
OXOOXX OX 	 0.0
```

In [ ]:
# Demo a single game
agent.demo_game(verbose=True)

In this lab, what does epsilon do?

YOUR ANSWER HERE

In this lab, why is `alpha = 1`?

YOUR ANSWER HERE

Summary
-----

- Even though tic-tac-toe is a simple game, you don't need to be able to explicitly define the optimal strategy to write an agent that can discover it.